In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
predictionFeature = 'SP.ADO.TFRT'
X = pd.read_csv("data/xSave.csv", index_col=0)
y = pd.read_csv("data/ySave.csv", index_col=0) #, usecols=['YEAR_5'])
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))
X.head()

,COUNTRY,YEAR,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
0,AFG,1971,1.673333e+09,161.384702,3826000.0,1224.516,83.05,70.20,3124500.0,60.38,...,7.450,100000000.0,99900000.0,47.314314,42.712713,141000000.0,145386000.0,32.425405,57.253106,13.161423
1,AFG,1977,1.733333e+09,144.173944,4270000.0,1279.592,91.25,73.56,3337000.0,58.45,...,7.449,306000000.0,313137000.0,45.115397,35.590492,328000000.0,458652000.0,45.553055,45.343964,21.467268
2,AFG,1978,2.155555e+09,175.027098,4351000.0,1301.915,94.92,76.02,3342000.0,61.00,...,7.449,321000000.0,269156000.0,43.718141,27.069432,395000000.0,450516000.0,53.467801,34.542835,21.696969
3,AFG,1979,2.366667e+09,188.085136,4481000.0,1316.392,97.68,78.24,3404000.0,63.38,...,7.449,474000000.0,435015000.0,60.016551,18.152248,425000000.0,551216000.0,68.909647,24.561515,24.310830
4,AFG,1980,2.555556e+09,199.164809,4624000.0,1362.404,102.93,83.72,3394000.0,69.07,...,7.449,670000000.0,670474000.0,64.592065,19.471896,841000000.0,494754000.0,60.669343,32.162044,41.491344


In [2]:
y.head()

,YEAR_0,YEAR_1,YEAR_2,YEAR_3,YEAR_4
0,145.3210,146.2150,147.1090,148.0030,148.8970
1,150.8782,151.9654,153.0526,154.1398,155.2270
2,151.9654,153.0526,154.1398,155.2270,156.3758
3,153.0526,154.1398,155.2270,156.3758,157.5246
4,154.1398,155.2270,156.3758,157.5246,158.6734


In [3]:
# Can use OBSERVATION to get new y Data if we want to train on something else
pData = data[data.IndicatorCode == predictionFeature]
pData.head()

# If I do this I need to add some value to year and filter for things that dont have
# it (eg dont have 'SP.ADO.TFRT' 4 years from now, drop this observation)
# Also need to remove filtering code from other file
indices = [(pData.CountryCode == row[0]) & (pData.Year == int(row[1])) for row in X.values]
values = [pData[index].Value.values[0] for index in indices]
#y = pd.DataFrame(values, index=X.OBSERVATION.values, columns=['Value'])

In [4]:
from sklearn.cross_validation import train_test_split
X_data = X.drop(['COUNTRY', 'YEAR'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=0)

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
train = y_train.YEAR_4 
test = y_test.YEAR_4

In [7]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, train)
#cv_scores = cross_val_score(clf, X_test, test, cv=4)
#cv_scores
clf.score(X_test, test)

0.99368182314774856

In [8]:
from sklearn.svm import SVR
clf = SVR()
clf.fit(X_train, train)
clf.score(X_test, test)

0.87752602923588452

In [9]:
from sklearn.grid_search import GridSearchCV
param_grid = [
    {'C': [10], 'kernel': ['linear']},
    {'C': [10], 'gamma': [0.03], 'kernel': ['rbf']},
]
clf = GridSearchCV(SVR(), param_grid, cv=4)
clf = clf.fit(X_train, train)
clf = clf.best_estimator_
clf.score(X_test, test)

KeyboardInterrupt: 

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt
def testClf(clf, country):
    plotData = data[(data.IndicatorCode == predictionFeature) & (data.CountryCode == country)]
    

In [24]:
country = 'AFG'

# TODO tomorrow stop saving observation keys as one value country_year, save in two rows
indices = (X.COUNTRY == country)
plotPredictionData = y[indices]
plotData = X[indices]
plotData.head()


# for each row, get the predicted value from clf and plotPredictionData and add the year+prediction with IsPredicted=true
# add IsPredicted= false for all current rows

,COUNTRY,YEAR,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
0,AFG,1971,1.673333e+09,161.384702,3826000.0,1224.516,83.05,70.20,3124500.0,60.38,...,7.450,100000000.0,99900000.0,47.314314,42.712713,141000000.0,145386000.0,32.425405,57.253106,13.161423
1,AFG,1977,1.733333e+09,144.173944,4270000.0,1279.592,91.25,73.56,3337000.0,58.45,...,7.449,306000000.0,313137000.0,45.115397,35.590492,328000000.0,458652000.0,45.553055,45.343964,21.467268
2,AFG,1978,2.155555e+09,175.027098,4351000.0,1301.915,94.92,76.02,3342000.0,61.00,...,7.449,321000000.0,269156000.0,43.718141,27.069432,395000000.0,450516000.0,53.467801,34.542835,21.696969
3,AFG,1979,2.366667e+09,188.085136,4481000.0,1316.392,97.68,78.24,3404000.0,63.38,...,7.449,474000000.0,435015000.0,60.016551,18.152248,425000000.0,551216000.0,68.909647,24.561515,24.310830
4,AFG,1980,2.555556e+09,199.164809,4624000.0,1362.404,102.93,83.72,3394000.0,69.07,...,7.449,670000000.0,670474000.0,64.592065,19.471896,841000000.0,494754000.0,60.669343,32.162044,41.491344


In [26]:
plotData.IsPredicted = [Fal] * len(plotData)
plotData.head()

,COUNTRY,YEAR,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
0,AFG,1971,1.673333e+09,161.384702,3826000.0,1224.516,83.05,70.20,3124500.0,60.38,...,7.450,100000000.0,99900000.0,47.314314,42.712713,141000000.0,145386000.0,32.425405,57.253106,13.161423
1,AFG,1977,1.733333e+09,144.173944,4270000.0,1279.592,91.25,73.56,3337000.0,58.45,...,7.449,306000000.0,313137000.0,45.115397,35.590492,328000000.0,458652000.0,45.553055,45.343964,21.467268
2,AFG,1978,2.155555e+09,175.027098,4351000.0,1301.915,94.92,76.02,3342000.0,61.00,...,7.449,321000000.0,269156000.0,43.718141,27.069432,395000000.0,450516000.0,53.467801,34.542835,21.696969
3,AFG,1979,2.366667e+09,188.085136,4481000.0,1316.392,97.68,78.24,3404000.0,63.38,...,7.449,474000000.0,435015000.0,60.016551,18.152248,425000000.0,551216000.0,68.909647,24.561515,24.310830
4,AFG,1980,2.555556e+09,199.164809,4624000.0,1362.404,102.93,83.72,3394000.0,69.07,...,7.449,670000000.0,670474000.0,64.592065,19.471896,841000000.0,494754000.0,60.669343,32.162044,41.491344


In [36]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, train)
#cv_scores = cross_val_score(clf, X_test, test, cv=4)
#cv_scores
clf.score(X_test, test)

0.99368182314774856

In [48]:
new = plotData
predictions = []
#for row in new:
test = clf.predict(plotData.drop(['COUNTRY', 'YEAR'], axis=1))
for i in range(len(X.YEAR)):
    predictions.append([X.YEAR[i], test[i], True])
a = pd.DataFrame(predictions)
b = 

IndexError: index 11 is out of bounds for axis 0 with size 11

In [20]:
[True] * len(plotData)

[True, True, True, True, True, True, True, True, True, True, True]

In [30]:
plotData.IsPredicted[0]

True

In [45]:
X.YEAR[0]

1971

In [ ]:
X['SP.ADO.TFRT']

In [ ]:
graph = sb.pointplot(x='Year', y='Value', hue='CountryCode', markers='IsPredicted', linestyles='IsPredicted', data=plotData)